# **Business Optimisation Analysis of Chinook Digital Music Store**

Business optimisation is the process of improving the efficiency, productivity and performance of an organisation. It is the idea of consistently striving for better outcomes. This can apply both to internal operations and external products. Key elements of business optimisation include:

- Measurement of productivity, efficiency and performance
- Identifying areas for improvement
- Introducing new methods and processes
- Measuring and comparing results
- Repeating the cycle

Some examples of business optimisation include:

- Introducing new methods, practices and systems that reduce turnaround time
- Reducing costs while improving performance
- Automation of repetitive tasks
- Increasing sales through enhancing customer satisfaction etc.

Hence, in this project I have conducted an analysis for Chinook, an imaginary digital music store, to help optimise their business. The database has been collected from the <a href="https://github.com/lerocha/chinook-database" target="_blank">github repo</a> of <a href="https://github.com/lerocha" target="_blank">Luis Rocha</a>.

## \*\*Use Chinook Database\*\*

In [10]:
USE Chinook;
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Most Profitable Genres

The Chinook record store has just signed a deal with a new record label, and we need to select the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

- Regal _Hip-Hop_
- Red Tone _Punk_
- Meteor and the Girls _Pop_
- Slim Jim Bites _Blues_

Let's find the top genres in Chinook.

In [13]:
SELECT Genre.Name AS genre, SUM(InvoiceLine.Quantity) AS tracks_sold, SUM(InvoiceLine.UnitPrice * InvoiceLine.Quantity) AS total_sales_USD
FROM InvoiceLine
JOIN Track 
ON InvoiceLine.TrackId = Track.TrackId 
JOIN Genre 
ON Track.GenreId = Genre.GenreId
GROUP BY Genre.Name
ORDER BY tracks_sold DESC;

(24 rows affected)

Total execution time: 00:00:00.098

genre,tracks_sold,total_sales_USD
Rock,835,826.65
Latin,386,382.14
Metal,264,261.36
Alternative & Punk,244,241.56
Jazz,80,79.20
Blues,61,60.39
TV Shows,47,93.53
R&B/Soul,41,40.59
Classical,41,40.59
Reggae,30,29.70


Based on the sales of tracks across different genres, we should definitely add the album by Red Tone (Punk), and may also add the albums by Slim Jim Bites (Blues) and Meteor and the Girls (Pop).

## Performance of Sales Support Agents

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [16]:
WITH total_sales_rep AS 
(
    SELECT c.SupportRepId, SUM(i.Total) AS total_sales
    FROM Customer c
    LEFT JOIN Invoice i
    ON c.CustomerId = i.CustomerId 
    GROUP BY c.SupportRepId 
)

SELECT (e.FirstName + ' ' +  e.LastName) AS Name, e.Title, e.HireDate, ts.total_sales 
FROM Employee e
JOIN total_sales_rep ts
ON e.EmployeeId = ts.SupportRepId 
ORDER BY ts.total_sales DESC;

(3 rows affected)

Total execution time: 00:00:00.030

Name,Title,HireDate,total_sales
Jane Peacock,Sales Support Agent,2002-04-01 00:00:00.000,833.04
Margaret Park,Sales Support Agent,2003-05-03 00:00:00.000,775.40
Steve Johnson,Sales Support Agent,2003-10-17 00:00:00.000,720.16


While there is about 14% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.